In [1]:
import json
import pandas as pd


In [2]:
!git clone https://github.com/statsbomb/open-data.git


Cloning into 'open-data'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 13128 (delta 5), reused 11 (delta 1), pack-reused 13113
Receiving objects: 100% (13128/13128), 1.27 GiB | 30.66 MiB/s, done.
Resolving deltas: 100% (11690/11690), done.
Checking out files: 100% (1828/1828), done.


In [25]:
# Loading Competitions
comp = pd.read_json(r'open-data/data/competitions.json')
comp.head()

,competition_id,season_id,country_name,competition_name,competition_gender,season_name,match_updated,match_available
0,16,4,Europe,Champions League,male,2018/2019,2020-10-25T12:33:27.855343,2020-10-25T12:33:27.855343
1,16,1,Europe,Champions League,male,2017/2018,2020-07-29T05:00,2020-07-29T05:00
2,16,2,Europe,Champions League,male,2016/2017,2020-08-26T12:33:15.869622,2020-07-29T05:00
3,16,27,Europe,Champions League,male,2015/2016,2020-08-26T12:33:15.869622,2020-07-29T05:00
4,16,26,Europe,Champions League,male,2014/2015,2020-08-26T12:33:15.869622,2020-07-29T05:00


In [4]:
comp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   competition_id      37 non-null     int64 
 1   season_id           37 non-null     int64 
 2   country_name        37 non-null     object
 3   competition_name    37 non-null     object
 4   competition_gender  37 non-null     object
 5   season_name         37 non-null     object
 6   match_updated       37 non-null     object
 7   match_available     37 non-null     object
dtypes: int64(2), object(6)
memory usage: 2.4+ KB


In [5]:
# Selecting competition ID and season ID (Here it is Premier League and 02/03)
competition_id = 2
season_id = 44

In [7]:
print('open-data/data/'+str(competition_id)+'/'+str(season_id)+'.json')

open-data/data/2/44.json


In [13]:
# Loading all the matches in the selected season and league
matches = pd.read_json(r'open-data/data/matches/2/44.json')
matches.head()

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,last_updated,metadata,match_week,competition_stage,referee
0,3749257,2004-05-15,16:00:00.000,"{'competition_id': 2, 'country_name': 'England...","{'season_id': 44, 'season_name': '2003/2004'}","{'home_team_id': 1, 'home_team_name': 'Arsenal...","{'away_team_id': 22, 'away_team_name': 'Leices...",2,1,available,2020-08-30T08:12:14.579037,"{'data_version': '1.1.0', 'shot_fidelity_versi...",38,"{'id': 1, 'name': 'Regular Season'}","{'id': 1279, 'name': 'None'}"
1,3749246,2004-03-28,17:05:00.000,"{'competition_id': 2, 'country_name': 'England...","{'season_id': 44, 'season_name': '2003/2004'}","{'home_team_id': 1, 'home_team_name': 'Arsenal...","{'away_team_id': 39, 'away_team_name': 'Manche...",1,1,available,2020-07-29T05:00,"{'data_version': '1.1.0', 'shot_fidelity_versi...",30,"{'id': 1, 'name': 'Regular Season'}","{'id': 1279, 'name': 'None'}"
2,3749153,2004-01-10,16:00:00.000,"{'competition_id': 2, 'country_name': 'England...","{'season_id': 44, 'season_name': '2003/2004'}","{'home_team_id': 1, 'home_team_name': 'Arsenal...","{'away_team_id': 47, 'away_team_name': 'Middle...",4,1,available,2020-08-30T08:12:14.579037,"{'data_version': '1.1.0', 'shot_fidelity_versi...",21,"{'id': 1, 'name': 'Regular Season'}","{'id': 1279, 'name': 'None'}"
3,3749642,2004-02-28,16:00:00.000,"{'competition_id': 2, 'country_name': 'England...","{'season_id': 44, 'season_name': '2003/2004'}","{'home_team_id': 1, 'home_team_name': 'Arsenal...","{'away_team_id': 75, 'away_team_name': 'Charlt...",2,1,available,2020-07-29T05:00,"{'data_version': '1.1.0', 'shot_fidelity_versi...",27,"{'id': 1, 'name': 'Regular Season'}","{'id': 1279, 'name': 'None'}"
4,3749358,2003-08-24,17:05:00.000,"{'competition_id': 2, 'country_name': 'England...","{'season_id': 44, 'season_name': '2003/2004'}","{'home_team_id': 47, 'home_team_name': 'Middle...","{'away_team_id': 1, 'away_team_name': 'Arsenal...",0,4,available,2020-07-29T05:00,"{'data_version': '1.1.0', 'shot_fidelity_versi...",2,"{'id': 1, 'name': 'Regular Season'}","{'id': 1279, 'name': 'None'}"


In [26]:
# Selecting only the relevant columns and making the data usable by cleaning the data
matches['competition'] = matches['competition'].apply(lambda x : x['competition_name'])
matches['home_team'] = matches['home_team'].apply(lambda x : x['home_team_name'])
matches['away_team'] = matches['away_team'].apply(lambda x : x['away_team_name'])
matches['season'] = matches['season'].apply(lambda x : x['season_name'])
matches.head()

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,last_updated,metadata,match_week,competition_stage,referee,home_team_name,away_team_name
0,3749257,2004-05-15,16:00:00.000,Premier League,2003/2004,Arsenal,Leicester City,2,1,available,2020-08-30T08:12:14.579037,"{'data_version': '1.1.0', 'shot_fidelity_versi...",38,"{'id': 1, 'name': 'Regular Season'}","{'id': 1279, 'name': 'None'}",Arsenal,Leicester City
1,3749246,2004-03-28,17:05:00.000,Premier League,2003/2004,Arsenal,Manchester United,1,1,available,2020-07-29T05:00,"{'data_version': '1.1.0', 'shot_fidelity_versi...",30,"{'id': 1, 'name': 'Regular Season'}","{'id': 1279, 'name': 'None'}",Arsenal,Manchester United
2,3749153,2004-01-10,16:00:00.000,Premier League,2003/2004,Arsenal,Middlesbrough,4,1,available,2020-08-30T08:12:14.579037,"{'data_version': '1.1.0', 'shot_fidelity_versi...",21,"{'id': 1, 'name': 'Regular Season'}","{'id': 1279, 'name': 'None'}",Arsenal,Middlesbrough
3,3749642,2004-02-28,16:00:00.000,Premier League,2003/2004,Arsenal,Charlton Athletic,2,1,available,2020-07-29T05:00,"{'data_version': '1.1.0', 'shot_fidelity_versi...",27,"{'id': 1, 'name': 'Regular Season'}","{'id': 1279, 'name': 'None'}",Arsenal,Charlton Athletic
4,3749358,2003-08-24,17:05:00.000,Premier League,2003/2004,Middlesbrough,Arsenal,0,4,available,2020-07-29T05:00,"{'data_version': '1.1.0', 'shot_fidelity_versi...",2,"{'id': 1, 'name': 'Regular Season'}","{'id': 1279, 'name': 'None'}",Middlesbrough,Arsenal


In [28]:
# Dropping unwanted columns
matches = matches.drop(columns=['metadata','competition_stage','referee','home_team_name','away_team_name'])
matches.head()

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,last_updated,match_week
0,3749257,2004-05-15,16:00:00.000,Premier League,2003/2004,Arsenal,Leicester City,2,1,available,2020-08-30T08:12:14.579037,38
1,3749246,2004-03-28,17:05:00.000,Premier League,2003/2004,Arsenal,Manchester United,1,1,available,2020-07-29T05:00,30
2,3749153,2004-01-10,16:00:00.000,Premier League,2003/2004,Arsenal,Middlesbrough,4,1,available,2020-08-30T08:12:14.579037,21
3,3749642,2004-02-28,16:00:00.000,Premier League,2003/2004,Arsenal,Charlton Athletic,2,1,available,2020-07-29T05:00,27
4,3749358,2003-08-24,17:05:00.000,Premier League,2003/2004,Middlesbrough,Arsenal,0,4,available,2020-07-29T05:00,2


In [32]:
for index,row in matches.iterrows():
  print("The match between "+row['home_team']+" and "+row['away_team']+" finished "+str(row['home_score'])+" - "+str(row['away_score'])+" in Gameweek "+str(row['match_week'])+".")

The match between Arsenal and Leicester City finished 2 - 1 in Gameweek 38.
The match between Arsenal and Manchester United finished 1 - 1 in Gameweek 30.
The match between Arsenal and Middlesbrough finished 4 - 1 in Gameweek 21.
The match between Arsenal and Charlton Athletic finished 2 - 1 in Gameweek 27.
The match between Middlesbrough and Arsenal finished 0 - 4 in Gameweek 2.
The match between Leeds United and Arsenal finished 1 - 4 in Gameweek 11.
The match between Liverpool and Arsenal finished 1 - 2 in Gameweek 8.
The match between Arsenal and Chelsea finished 2 - 1 in Gameweek 9.
The match between Blackburn Rovers and Arsenal finished 0 - 2 in Gameweek 28.
The match between Aston Villa and Arsenal finished 0 - 2 in Gameweek 22.
The match between Arsenal and Newcastle United finished 3 - 2 in Gameweek 7.
The match between Arsenal and Manchester City finished 2 - 1 in Gameweek 23.
The match between Arsenal and Birmingham City finished 0 - 0 in Gameweek 35.
The match between Manch

In [37]:
# Getting Match ID to get the events data for the match
home_team = 'Arsenal'
away_team = 'Liverpool'

for index, team in matches.iterrows():
  if (team['home_team'] == home_team and team['away_team'] == away_team):
    print("Match ID for game between "+home_team+" and "+away_team+" is "+str(team['match_id']))
    match_id = team['match_id']

Match ID for game between Arsenal and Liverpool is 3749448


In [46]:
with open ('open-data/data/events/'+str(match_id)+'.json') as datafile:
  data = json.load(datafile)

In [45]:
events_df = pd.json_normalize(data, sep='_')
events_df.head()

,id,index,period,timestamp,minute,second,possession,duration,type_id,type_name,possession_team_id,possession_team_name,play_pattern_id,play_pattern_name,team_id,team_name,tactics_formation,tactics_lineup,related_events,location,player_id,player_name,position_id,position_name,pass_recipient_id,pass_recipient_name,pass_length,pass_angle,pass_height_id,pass_height_name,pass_end_location,pass_body_part_id,pass_body_part_name,pass_type_id,pass_type_name,carry_end_location,pass_outcome_id,pass_outcome_name,ball_receipt_outcome_id,ball_receipt_outcome_name,...,pass_assisted_shot_id,pass_goal_assist,shot_key_pass_id,goalkeeper_outcome_id,goalkeeper_outcome_name,goalkeeper_technique_id,goalkeeper_technique_name,clearance_right_foot,clearance_left_foot,ball_recovery_recovery_failure,foul_won_defensive,foul_committed_advantage,foul_won_advantage,duel_outcome_id,duel_outcome_name,pass_aerial_won,miscontrol_aerial_won,shot_first_time,pass_shot_assist,shot_aerial_won,shot_one_on_one,goalkeeper_body_part_id,goalkeeper_body_part_name,pass_outswinging,foul_committed_type_id,foul_committed_type_name,foul_committed_card_id,foul_committed_card_name,pass_cut_back,dribble_overrun,pass_through_ball,pass_miscommunication,substitution_outcome_id,substitution_outcome_name,substitution_replacement_id,substitution_replacement_name,bad_behaviour_card_id,bad_behaviour_card_name,block_offensive,shot_redirect
0,ffdbffbf-aad2-4694-a079-8ab4e0f20769,1,1,00:00:00.000,0,0,1,0.00000,35,Starting XI,1,Arsenal,1,Regular Play,1,Arsenal,442.0,"[{'player': {'id': 20015, 'name': 'Jens Lehman...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8ab2e7df-7642-4f9b-ad8e-4da95dfe7fa2,2,1,00:00:00.000,0,0,1,0.00000,35,Starting XI,1,Arsenal,1,Regular Play,24,Liverpool,442.0,"[{'player': {'id': 34390, 'name': 'Jerzy Dudek...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,c77df932-7a63-4d12-99de-542ee0115a37,3,1,00:00:00.000,0,0,1,0.00000,18,Half Start,1,Arsenal,1,Regular Play,24,Liverpool,NaN,NaN,[158f68e3-189a-498f-93b9-412a30ec4287],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,158f68e3-189a-498f-93b9-412a30ec4287,4,1,00:00:00.000,0,0,1,0.00000,18,Half Start,1,Arsenal,1,Regular Play,1,Arsenal,NaN,NaN,[c77df932-7a63-4d12-99de-542ee0115a37],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,255af120-7cad-463c-9d2f-884784fca4c6,5,1,00:00:00.278,0,0,2,0.44968,30,Pass,24,Liverpool,9,From Kick Off,24,Liverpool,NaN,NaN,[a82c7ba3-9e03-4ffa-99de-a9e1a566cf7b],"[61.0, 40.1]",19109.0,Steven Gerrard,11.0,Left Defensive Midfield,15665.0,Michael Owen,3.883298,0.968509,1.0,Ground Pass,"[63.2, 43.3]",40.0,Right Foot,65.0,Kick Off,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
passes = events_df[events_df['type_name']=='Pass']
passes.reset_index(inplace=True)
passes.head()

,level_0,id,index,period,timestamp,minute,second,possession,duration,type_id,type_name,possession_team_id,possession_team_name,play_pattern_id,play_pattern_name,team_id,team_name,tactics_formation,tactics_lineup,related_events,location,player_id,player_name,position_id,position_name,pass_recipient_id,pass_recipient_name,pass_length,pass_angle,pass_height_id,pass_height_name,pass_end_location,pass_body_part_id,pass_body_part_name,pass_type_id,pass_type_name,carry_end_location,pass_outcome_id,pass_outcome_name,ball_receipt_outcome_id,...,pass_assisted_shot_id,pass_goal_assist,shot_key_pass_id,goalkeeper_outcome_id,goalkeeper_outcome_name,goalkeeper_technique_id,goalkeeper_technique_name,clearance_right_foot,clearance_left_foot,ball_recovery_recovery_failure,foul_won_defensive,foul_committed_advantage,foul_won_advantage,duel_outcome_id,duel_outcome_name,pass_aerial_won,miscontrol_aerial_won,shot_first_time,pass_shot_assist,shot_aerial_won,shot_one_on_one,goalkeeper_body_part_id,goalkeeper_body_part_name,pass_outswinging,foul_committed_type_id,foul_committed_type_name,foul_committed_card_id,foul_committed_card_name,pass_cut_back,dribble_overrun,pass_through_ball,pass_miscommunication,substitution_outcome_id,substitution_outcome_name,substitution_replacement_id,substitution_replacement_name,bad_behaviour_card_id,bad_behaviour_card_name,block_offensive,shot_redirect
0,4,255af120-7cad-463c-9d2f-884784fca4c6,5,1,00:00:00.278,0,0,2,0.449680,30,Pass,24,Liverpool,9,From Kick Off,24,Liverpool,NaN,NaN,[a82c7ba3-9e03-4ffa-99de-a9e1a566cf7b],"[61.0, 40.1]",19109.0,Steven Gerrard,11.0,Left Defensive Midfield,15665.0,Michael Owen,3.883298,0.968509,1.0,Ground Pass,"[63.2, 43.3]",40.0,Right Foot,65.0,Kick Off,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7,267f5c32-b8af-49e5-9c04-9307f24c9c3c,8,1,00:00:00.848,0,0,2,0.997256,30,Pass,24,Liverpool,9,From Kick Off,24,Liverpool,NaN,NaN,[cd1c2520-a489-4e05-af9f-2398702c47ae],"[64.1, 44.3]",15665.0,Michael Owen,24.0,Left Center Forward,38522.0,Dietmar Hamann,16.790474,-2.925508,1.0,Ground Pass,"[47.7, 40.7]",40.0,Right Foot,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10,34373762-8408-4e56-bef9-f782e0e797d4,11,1,00:00:02.918,0,2,2,1.394844,30,Pass,24,Liverpool,9,From Kick Off,24,Liverpool,NaN,NaN,[1f4d3c52-eebc-40e3-b723-628c251cc4cb],"[46.5, 43.2]",38522.0,Dietmar Hamann,9.0,Right Defensive Midfield,34388.0,Jamie Carragher,33.130802,1.820899,2.0,Low Pass,"[38.3, 75.3]",40.0,Right Foot,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13,07c754e8-840c-4711-98f3-15c50c070e9c,14,1,00:00:06.802,0,6,2,2.532285,30,Pass,24,Liverpool,9,From Kick Off,24,Liverpool,NaN,NaN,[e2cfdb94-3598-4860-8f77-6e1c922c6769],"[42.2, 72.7]",34388.0,Jamie Carragher,2.0,Right Back,NaN,NaN,55.374092,-0.177915,3.0,High Pass,"[96.7, 62.9]",40.0,Right Foot,NaN,NaN,NaN,9.0,Incomplete,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14,e2cfdb94-3598-4860-8f77-6e1c922c6769,15,1,00:00:09.334,0,9,2,0.925050,30,Pass,24,Liverpool,9,From Kick Off,1,Arsenal,NaN,NaN,"[07c754e8-840c-4711-98f3-15c50c070e9c, 961c92f...","[23.4, 17.2]",15637.0,"Sulzeer Jeremiah ""Sol"" Campbell",5.0,Left Center Back,15515.0,Patrick Vieira,14.509652,-0.411227,3.0,High Pass,"[36.7, 11.4]",37.0,Head,66.0,Recovery,NaN,9.0,Incomplete,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
